In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
prov_ids = pd.read_csv('./../datos/info/radio_ref.csv')[['PROV_REF_ID', 'IDPROV']].drop_duplicates().rename(columns = {'IDPROV': 'in1_prov', 'PROV_REF_ID': 'distrito_id'}).reset_index(drop = True)
prov_ids['in1_prov'] = prov_ids['in1_prov'].astype(str).str.zfill(2)
prov_ids.head()

,distrito_id,in1_prov
0,1,02
1,2,06
2,3,10
3,4,14
4,5,18


In [3]:
eleccion_tags = pd.read_csv('./../datos/BD/eleccion_tags.csv')
eleccion_tags.head()

,eleccion_id,año,eleccion_tipo,recuento_tipo,padron_tipo,eleccion_tag
0,1,2011,PASO,PROVISORIO,NORMAL,PASO11n
1,10,2017,PASO,PROVISORIO,NORMAL,PASO17n
2,9,2017,PASO,PROVISORIO,COMANDO,PASO17c
3,8,2017,GENERAL,PROVISORIO,NORMAL,GRAL17n
4,7,2017,GENERAL,PROVISORIO,COMANDO,GRAL17c


In [4]:
cargo = pd.read_csv('./../datos/BD/cargo_tags.csv')
cargo

,cargo_id,cargo_nombre,cargo_tag
0,3,Diputado Nacional,DN
1,1,Presidente,PR
2,4,Gobernador,GB
3,6,Diputado Provincial,DP
4,2,Senador Nacional,SN
5,5,Senador Provincial,SP
6,10,Concejales,CO
7,9,Parlamentarios Mercosur Provinciales,MP
8,8,Parlamentarios Mercosur Nacionales,MN
9,7,Concejal,IN


## Provincias

In [5]:
# provs = gpd.read_file('./../datos/info/provincia.json')[['in1', 'geometry']].rename(columns = {'in1': 'in1_prov'})
provs = gpd.read_file('./../datos/geojson/provs_simplified_100.geojson')[['in1', 'geometry']].rename(columns = {'in1': 'in1_prov'})

In [6]:

data_prov = pd.read_csv('./../datos/out/votos_prov.csv').merge(prov_ids)
data_prov['fuerza'] = data_prov['fuerza'].fillna('NO POSITIVOS')
data_prov = data_prov.merge(eleccion_tags).merge(cargo)


In [7]:
data_prov_cargo = data_prov.copy()

# data_prov_long.groupby(['eleccion_tag', 'cargo_tag', 'in1_prov'])['fuerza'].agg(['unique', 'nunique'])

In [8]:
data_prov_cargo

,eleccion_id,cargo_id,fuerza,distrito_id,votos_cantidad,in1_prov,año,eleccion_tipo,recuento_tipo,padron_tipo,eleccion_tag,cargo_nombre,cargo_tag
0,0,1,NO POSITIVOS,1,52388,02,2011,GENERAL,PROVISORIO,NORMAL,GRAL11n,Presidente,PR
1,0,1,IZQUIERDA,1,61055,02,2011,GENERAL,PROVISORIO,NORMAL,GRAL11n,Presidente,PR
2,0,1,OTROS,1,271791,02,2011,GENERAL,PROVISORIO,NORMAL,GRAL11n,Presidente,PR
3,0,1,PERON,1,653520,02,2011,GENERAL,PROVISORIO,NORMAL,GRAL11n,Presidente,PR
4,0,1,PROVINCIAL,1,184116,02,2011,GENERAL,PROVISORIO,NORMAL,GRAL11n,Presidente,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,6,9,PERON,23,428039,90,2015,PASO,PROVISORIO,NORMAL,PASO15n,Parlamentarios Mercosur Provinciales,MP
4149,6,9,PRO,23,193955,90,2015,PASO,PROVISORIO,NORMAL,PASO15n,Parlamentarios Mercosur Provinciales,MP
4150,6,9,NO POSITIVOS,24,22550,94,2015,PASO,PROVISORIO,NORMAL,PASO15n,Parlamentarios Mercosur Provinciales,MP
4151,6,9,OTROS,24,35044,94,2015,PASO,PROVISORIO,NORMAL,PASO15n,Parlamentarios Mercosur Provinciales,MP


In [9]:
data_prov_cargo = data_prov_cargo.loc[data_prov_cargo.cargo_tag.isin(['PR', 'DN'])]
info = data_prov_cargo.groupby(['eleccion_tag', 'cargo_tag']).agg({'votos_cantidad': 'sum', 'fuerza': ['unique', 'nunique']})
info = info.loc[info[('votos_cantidad', 'sum')] > 10e6]
eleccion_cargo = info.sort_index().reset_index()[info.index.names]

In [10]:
eleccion_cargo.eleccion_tag.unique()

array(['BLTG15n', 'GRAL11n', 'GRAL13n', 'GRAL15n', 'GRAL17n', 'GRAL19n',
       'PASO11n', 'PASO13n', 'PASO15n', 'PASO17n', 'PASO19n', 'PASO21n'],
      dtype=object)

In [11]:
eleccion_cargo.cargo_tag.unique()

array(['PR', 'DN'], dtype=object)

In [12]:
# info.sort_values(by = [('votos_cantidad', 'sum')], ascending = False)
data_prov_cargo.fuerza.unique()

array(['NO POSITIVOS', 'IZQUIERDA', 'OTROS', 'PERON', 'PROVINCIAL',
       'SOCIALISTA', 'UCR', 'PRO', 'MASSA'], dtype=object)

In [13]:
import itertools

# Define the series with two level indices and values as lists of strings
series = info['fuerza']['unique'].sort_index()

# # Generate the base names by combining the strings
# base_names = ['_'.join(items) for items in itertools.product(series.index.levels[0], series.values)]

# # Print the base names
# for base_name in base_names:
#     print(base_name)

In [14]:
pd.options.display.max_colwidth = 300

info['fuerza']['unique'].sort_index()

# eleccion_tag  cargo_tag
# BLTG15n       PR                                                                 [NO POSITIVOS, PERON, PRO]
# GRAL11n       DN                  [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, PROVINCIAL, SOCIALISTA, UCR]
#               PR                       [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PROVINCIAL, SOCIALISTA, UCR]
# GRAL13n       DN           [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, MASSA, PROVINCIAL, SOCIALISTA, UCR]

eleccion_tag  cargo_tag
BLTG15n       PR                                                                 [NO POSITIVOS, PERON, PRO]
GRAL11n       DN                  [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, PROVINCIAL, SOCIALISTA, UCR]
              PR                       [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PROVINCIAL, SOCIALISTA, UCR]
GRAL13n       DN           [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, MASSA, PROVINCIAL, SOCIALISTA, UCR]
GRAL15n       DN           [NO POSITIVOS, IZQUIERDA, MASSA, OTROS, PERON, PRO, SOCIALISTA, PROVINCIAL, UCR]
              PR                            [NO POSITIVOS, IZQUIERDA, MASSA, OTROS, PERON, PRO, PROVINCIAL]
GRAL17n       DN                [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, MASSA, PROVINCIAL, SOCIALISTA]
GRAL19n       DN                       [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO, PROVINCIAL, SOCIALISTA]
              PR                                               [NO POSITIVOS, IZQUIERDA, OTROS, PERON, PRO]
PASO

In [15]:
# Group by 'eleccion_tag', 'cargo_tag', and 'in1_prov', and calculate the sum of 'votos_cantidad', divide for PCT
sum_votes = data_prov.groupby(['eleccion_tag', 'cargo_tag', 'in1_prov'])['votos_cantidad'].transform('sum')
data_prov['votos_porcentaje'] = data_prov['votos_cantidad'] / sum_votes


In [16]:

data_prov = data_prov.set_index(['in1_prov', 'eleccion_tag', 'cargo_tag', 'fuerza'])


data_prov_table_cnt = data_prov['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])
data_prov_table_pct = data_prov['votos_porcentaje'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])

data_prov_geoms_cnt = gpd.GeoDataFrame(data_prov_table_cnt.merge(provs, left_index = True, right_on='in1_prov')).set_index('in1_prov')
data_prov_geoms_pct = gpd.GeoDataFrame(data_prov_table_pct.merge(provs, left_index = True, right_on='in1_prov')).set_index('in1_prov')

# data_prov_geoms_pct.head()

/tmp/ipykernel_19302/4027131826.py:7: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_prov_geoms_cnt = gpd.GeoDataFrame(data_prov_table_cnt.merge(provs, left_index = True, right_on='in1_prov')).set_index('in1_prov')
/tmp/ipykernel_19302/4027131826.py:8: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_prov_geoms_pct = gpd.GeoDataFrame(data_prov_table_pct.merge(provs, left_index = True, right_on='in1_prov')).set_index('in1_prov')


In [17]:
data_prov_geoms_cnt.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_prov_geoms_cnt.columns]
data_prov_geoms_pct.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_prov_geoms_pct.columns]

data_prov_geoms_cnt.fillna(0).to_file('./../datos/geojson/votos_cnt_prov.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON
data_prov_geoms_pct.fillna(0).to_file('./../datos/geojson/votos_pct_prov.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON


## Circuitos

In [18]:

data_circ = pd.read_csv('./../datos/out/votos_circ.csv').merge(prov_ids)
data_circ['fuerza'] = data_circ['fuerza'].fillna('NO POSITIVOS')

# Guardar ref provs - dptos - circs
dist_secc_circ = data_circ.copy() # save for later

# data_circ
data_circ = data_circ.merge(eleccion_tags).merge(cargo)


# Group by 'eleccion_tag', 'cargo_tag', and 'in1_prov', and calculate the sum of 'votos_cantidad', divide for PCT
sum_votes = data_circ.groupby(['eleccion_tag', 'cargo_tag', 'in1_prov', 'circuito_id'])['votos_cantidad'].transform('sum')
data_circ['votos_porcentaje'] = data_circ['votos_cantidad'] / sum_votes


data_circ = data_circ.set_index(['distrito_id', 'circuito_id', 'eleccion_tag', 'cargo_tag', 'fuerza'])


data_circ_table_cnt = data_circ['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])
data_circ_table_pct = data_circ['votos_porcentaje'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])


In [19]:
# circuitos = gpd.read_file('/home/matias/repos/CNE-INDEC-georef/mapaelectoral/circuitos-CNE-TTGL.geojson')
# circuitos = gpd.read_file('./../datos/info/circuitos_dpto_prov.geojson').merge(prov_ids)
circuitos = gpd.read_file('./../datos/geojson/circs_simplified_1.geojson').merge(prov_ids)
circuitos['circuito_id'] = circuitos['circuito'].str.zfill(6)

CNE_in1 = circuitos[['distrito_id', 'circuito_id', 'in1_prov', 'in1_dpto']]
dist_secc_in1 = dist_secc_circ.merge(CNE_in1, on = ['distrito_id', 'circuito_id', 'in1_prov'])
dist_secc_in1 = dist_secc_in1.groupby(['distrito_id', 'seccion_id', 'in1_prov', 'in1_dpto'])['votos_cantidad'].sum().reset_index()

circuitos = circuitos[['distrito_id', 'in1_prov', 'circuito_id', 'geometry']]
circuitos.tail()

,distrito_id,in1_prov,circuito_id,geometry
5524,24,94,000014,"MULTIPOLYGON Z (((-67.63611 -53.80997 0.00000, -67.63299 -53.81126 0.00000, -67.63765 -53.81574 0.00000, -67.64869 -53.82223 0.00000, -67.69263 -53.82500 0.00000, -67.68215 -53.81607 0.00000, -67.73156 -53.81110 0.00000, -67.72722 -53.80976 0.00000, -67.72112 -53.80836 0.00000, -67.71554 -53.807..."
5525,24,94,000016,"POLYGON Z ((-68.30787 -53.32125 0.00000, -68.30604 -53.32109 0.00000, -68.30642 -53.35930 0.00000, -68.30554 -53.48470 0.00000, -68.31122 -53.48644 0.00000, -68.32452 -53.48775 0.00000, -68.32932 -53.48710 0.00000, -68.33411 -53.48731 0.00000, -68.33913 -53.48993 0.00000, -68.35200 -53.49189 0.0..."
5526,24,94,000021,"POLYGON Z ((-67.16125 -54.47838 0.00000, -67.13859 -54.55013 0.00000, -67.22534 -54.55306 0.00000, -67.24502 -54.48237 0.00000, -67.16125 -54.47838 0.00000))"
5527,24,94,000210,"POLYGON Z ((-67.73934 -53.77538 0.00000, -67.73960 -53.76105 0.00000, -67.73596 -53.76096 0.00000, -67.73569 -53.76400 0.00000, -67.73372 -53.76501 0.00000, -67.73597 -53.76656 0.00000, -67.73353 -53.76769 0.00000, -67.73522 -53.76881 0.00000, -67.71734 -53.77798 0.00000, -67.73932 -53.77926 0.0..."
5528,24,94,000221,"POLYGON Z ((-67.73940 -53.75336 0.00000, -67.77787 -53.73761 0.00000, -67.78147 -53.73597 0.00000, -67.79492 -53.72837 0.00000, -67.80392 -53.72022 0.00000, -67.81279 -53.71551 0.00000, -67.83032 -53.70949 0.00000, -67.84095 -53.70225 0.00000, -67.84635 -53.69461 0.00000, -67.85268 -53.69048 0.0..."


In [20]:

data_circ_geoms_cnt = gpd.GeoDataFrame(data_circ_table_cnt.merge(circuitos, left_index = True, right_on=['distrito_id', 'circuito_id'])).set_index(['distrito_id', 'circuito_id'])
data_circ_geoms_pct = gpd.GeoDataFrame(data_circ_table_pct.merge(circuitos, left_index = True, right_on=['distrito_id', 'circuito_id'])).set_index(['distrito_id', 'circuito_id'])

data_circ_geoms_cnt.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_circ_geoms_cnt.columns]
data_circ_geoms_pct.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_circ_geoms_pct.columns]

# data_circ_geoms_cnt.head(1000).to_file('./../datos/geojson/votos_cnt_circ_lite.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON
# data_circ_geoms_pct.head(1000).to_file('./../datos/geojson/votos_pct_circ_lite.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON

data_circ_geoms_cnt.to_file('./../datos/geojson/votos_cnt_circ.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON
data_circ_geoms_pct.to_file('./../datos/geojson/votos_pct_circ.geojson', driver='GeoJSON') # Save the GeoDataFrame to GeoJSON

# data_prov_geoms_pct.head()

/tmp/ipykernel_19302/1574007664.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_circ_geoms_cnt = gpd.GeoDataFrame(data_circ_table_cnt.merge(circuitos, left_index = True, right_on=['distrito_id', 'circuito_id'])).set_index(['distrito_id', 'circuito_id'])
/tmp/ipykernel_19302/1574007664.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_circ_geoms_pct = gpd.GeoDataFrame(data_circ_table_pct.merge(circuitos, left_index = True, right_on=['distrito_id', 'circuito_id'])).set_index(['distrito_id', 'circuito_id'])


In [21]:
## DEBUG de merge geometria circuitos

# ## Por problemas en merge de circuito, lo que se mantiene es el 96.66% de los votos total
# 100*data_circ_geoms.dropna().votos_cantidad.sum()/data_circ_geoms.votos_cantidad.sum()

## Verificar los casos con mas votos perdidos por el merge
# data_circ_geoms.loc[data_circ_geoms.geometry.isna()].groupby('distrito_id')['votos_cantidad'].sum().sort_values()
# data_circ_geoms.loc[data_circ_geoms.geometry.isna()].groupby(['distrito_id', 'circuito_id'])['votos_cantidad'].sum().sort_values().tail(10)

# ## Excepto por 4 casos puntuales, se puede decir que el id de circuito es unico en cada provincia. Es decir se puede unir por provincia y circuito.
# circuitos.groupby(['in1_prov', 'circuito']).in1_dpto.nunique().sort_values(ascending=False).head()
# # in1_prov  circuito
# # 14        sindatos    3
# #           zonagris    3
# # 22        00072       2
# # 30        00303       2
# # 58        01550       1

### Referencia seccion_id - in1_dpto

Se obtiene tomando para cada seccion_id el in1_dpto que mas votos acumula. Se asume que el in1_dpto que mas se repite es el correcto.


In [22]:

# take n largest
# largest_intersections = intersections.groupby(['codprov', 'coddepto', 'circuito']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop=True)

largest = dist_secc_in1.groupby(['distrito_id', 'seccion_id']).apply(lambda x: x.nlargest(1, 'votos_cantidad')).reset_index(drop=True)

largest[['distrito_id', 'seccion_id', 'in1_prov', 'in1_dpto']].to_csv('./../datos/info/seccion_dpto.csv', index = False)

## Departamentos (Secciones)

In [23]:
dptos = gpd.read_file('./../datos/geojson/dptos_simplified_2.geojson')[['in1', 'geometry']].rename(columns = {'in1': 'in1_dpto'})
dptos['in1_dpto'] = dptos['in1_dpto'].astype(int)
# dptos['in1_prov'] = dptos['in1_dpto'].astype(str).str[:2].astype(int)
dptos.head()

,in1_dpto,geometry
0,30015,"POLYGON ((-58.38761 -30.82889, -58.38711 -30.83035, -58.38629 -30.83156, -58.38421 -30.83035, -58.38427 -30.82917, -58.38383 -30.82897, -58.38237 -30.82911, -58.38067 -30.83054, -58.37921 -30.83056, -58.37771 -30.83004, -58.37708 -30.83060, -58.37743 -30.83221, -58.37707 -30.83274, -58.37544 -30..."
1,30035,"POLYGON ((-58.96265 -30.60344, -58.96104 -30.60525, -58.95838 -30.60571, -58.95797 -30.60663, -58.95701 -30.60748, -58.95574 -30.60744, -58.95448 -30.60633, -58.95368 -30.60691, -58.95383 -30.60828, -58.95354 -30.60917, -58.95210 -30.61026, -58.95083 -30.61028, -58.95031 -30.60997, -58.95001 -30..."
2,30056,"POLYGON ((-59.06547 -32.48629, -59.06444 -32.48675, -59.06367 -32.48661, -59.06291 -32.48582, -59.06142 -32.48542, -59.05723 -32.48683, -59.05614 -32.48804, -59.05484 -32.48873, -59.05324 -32.48898, -59.05211 -32.48845, -59.05113 -32.48897, -59.05031 -32.48835, -59.04674 -32.49037, -59.04662 -32..."
3,30063,"POLYGON ((-58.43841 -33.51761, -58.44346 -33.53784, -58.44408 -33.53964, -58.44501 -33.54105, -58.44769 -33.54360, -58.47775 -33.56409, -58.48004 -33.56597, -58.49376 -33.58146, -58.49467 -33.58369, -58.49464 -33.58719, -58.48493 -33.61455, -58.47674 -33.64627, -58.47535 -33.65024, -58.47349 -33..."
4,30105,"POLYGON ((-60.16654 -32.27211, -60.16239 -32.26576, -60.14369 -32.27519, -60.13687 -32.26628, -60.13256 -32.26812, -60.13123 -32.26897, -60.12970 -32.27041, -60.11953 -32.28455, -60.11886 -32.28691, -60.11892 -32.28916, -60.11988 -32.29655, -60.11840 -32.30029, -60.11586 -32.30344, -60.09799 -32..."


In [24]:
seccion_dpto = pd.read_csv('./../datos/info/seccion_dpto.csv')
seccion_dpto.head()

,distrito_id,seccion_id,in1_prov,in1_dpto
0,1,1,2,2007
1,1,2,2,2014
2,1,3,2,2021
3,1,4,2,2028
4,1,5,2,2035


In [25]:
seccion_dpto.merge(dptos).head()

,distrito_id,seccion_id,in1_prov,in1_dpto,geometry
0,1,1,2,2007,"POLYGON ((-58.38609 -34.57823, -58.38320 -34.57833, -58.37980 -34.58018, -58.37613 -34.57997, -58.37509 -34.58054, -58.37387 -34.57871, -58.36789 -34.57823, -58.36532 -34.57963, -58.36525 -34.58016, -58.37063 -34.58059, -58.37062 -34.58082, -58.36904 -34.58174, -58.36164 -34.58116, -58.36142 -34..."
1,1,2,2,2014,"POLYGON ((-58.38359 -34.57339, -58.38180 -34.57440, -58.37552 -34.57392, -58.37077 -34.57662, -58.37076 -34.57708, -58.37610 -34.57752, -58.37434 -34.57871, -58.37387 -34.57871, -58.37509 -34.58054, -58.37613 -34.57997, -58.37980 -34.58018, -58.38354 -34.57825, -58.38983 -34.57822, -58.39161 -34..."
2,1,3,2,2021,"POLYGON ((-58.41026 -34.59811, -58.40447 -34.59805, -58.40198 -34.59939, -58.39869 -34.59978, -58.39290 -34.59966, -58.39235 -34.60317, -58.39177 -34.61130, -58.39114 -34.62727, -58.40833 -34.62953, -58.41175 -34.63037, -58.41230 -34.62305, -58.41201 -34.62299, -58.41240 -34.62190, -58.41284 -34..."
3,1,4,2,2028,"POLYGON ((-58.35517 -34.61945, -58.35404 -34.61972, -58.35349 -34.61942, -58.35266 -34.61979, -58.35253 -34.61956, -58.35189 -34.61964, -58.35158 -34.61925, -58.34941 -34.61975, -58.34774 -34.61896, -58.34649 -34.61911, -58.34526 -34.61875, -58.34461 -34.61896, -58.34135 -34.61843, -58.33837 -34..."
4,1,5,2,2035,"POLYGON ((-58.42271 -34.59773, -58.41189 -34.59802, -58.41387 -34.60726, -58.41407 -34.60793, -58.41463 -34.60821, -58.41443 -34.61075, -58.41284 -34.61413, -58.41149 -34.63287, -58.41045 -34.63525, -58.41123 -34.63812, -58.42346 -34.64022, -58.42577 -34.63130, -58.42922 -34.61512, -58.42999 -34..."


In [26]:

# data_circ = data_circ.merge(eleccion_tags).set_index(['in1_prov', 'circuito_id', 'eleccion_tag', 'cargo_tag', 'fuerza'])
# data_circ_table = data_circ['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])

# data_circ_table

In [27]:
data_secc = pd.read_csv('./../datos/out/votos_secc.csv').merge(seccion_dpto.merge(dptos), how = 'left') ## Solo se pierde la seccion 24-3 que es la Antartida Argentina
data_secc['in1_prov'] = data_secc['in1_prov'].fillna(24).astype(int)#.astype(str).str.zfill(2)
data_secc['in1_dpto'] = data_secc['in1_dpto'].fillna(24003).astype(int)#.astype(str).str.zfill(5)
data_secc['fuerza'] = data_secc['fuerza'].fillna('NO POSITIVOS')

# .set_index(['in1_prov', 'circuito_id', 'eleccion_tag', 'cargo_tag', 'fuerza'])
# # data_circ_table = data_circ['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])

data_secc = data_secc.merge(eleccion_tags).merge(cargo)

# data_secc = data_secc.set_index(['in1_prov', 'in1_dpto', 'eleccion_tag', 'cargo_tag', 'fuerza'])
# data_secc_table = data_secc['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])

## Replace by the following, as there are a few which are duplicate for some reason..
data_secc = data_secc.groupby(['in1_prov', 'in1_dpto', 'eleccion_tag', 'cargo_tag', 'fuerza'])['votos_cantidad'].sum().reset_index()



In [30]:

# Group by 'eleccion_tag', 'cargo_tag', and 'in1_prov', and calculate the sum of 'votos_cantidad', divide for PCT
sum_votes = data_secc.groupby(['eleccion_tag', 'cargo_tag', 'in1_prov', 'in1_dpto'])['votos_cantidad'].transform('sum')
data_secc['votos_porcentaje'] = data_secc['votos_cantidad'] / sum_votes
data_secc_pcts = data_secc.copy()

## Guardar para sabes culaes son los valores tipicos y de ahi definir escalas de colores.
data_secc_pcts.to_csv('./../datos/info/data_secc_pct_vals.csv', index = False)

data_secc = data_secc.set_index(['in1_prov', 'in1_dpto', 'eleccion_tag', 'cargo_tag', 'fuerza'])


data_secc_table_cnt = data_secc['votos_cantidad'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])
data_secc_table_pct = data_secc['votos_porcentaje'].unstack(['eleccion_tag', 'cargo_tag', 'fuerza'])


### Check pct values por fuerza

In [32]:

data_secc_geoms_cnt = gpd.GeoDataFrame(data_secc_table_cnt.merge(seccion_dpto.merge(dptos), left_index = True, right_on=['in1_prov', 'in1_dpto'])).set_index(['distrito_id', 'seccion_id'])
data_secc_geoms_pct = gpd.GeoDataFrame(data_secc_table_pct.merge(seccion_dpto.merge(dptos), left_index = True, right_on=['in1_prov', 'in1_dpto'])).set_index(['distrito_id', 'seccion_id'])

data_secc_geoms_cnt.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_secc_geoms_cnt.columns]
data_secc_geoms_pct.columns = [str(col).replace("(", "").replace(")", "").replace("'", "").replace(", ", "-") for col in data_secc_geoms_pct.columns]

data_secc_geoms_cnt.to_file('./../datos/geojson/votos_cnt_secc.geojson', driver = 'GeoJSON')
data_secc_geoms_pct.to_file('./../datos/geojson/votos_pct_secc.geojson', driver = 'GeoJSON') # Save the GeoDataFrame to GeoJSON

# data_prov_geoms_pct.head()

/tmp/ipykernel_19302/1403680475.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_secc_geoms_cnt = gpd.GeoDataFrame(data_secc_table_cnt.merge(seccion_dpto.merge(dptos), left_index = True, right_on=['in1_prov', 'in1_dpto'])).set_index(['distrito_id', 'seccion_id'])
/tmp/ipykernel_19302/1403680475.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (3 levels on the left, 1 on the right)
  data_secc_geoms_pct = gpd.GeoDataFrame(data_secc_table_pct.merge(seccion_dpto.merge(dptos), left_index = True, right_on=['in1_prov', 'in1_dpto'])).set_index(['distrito_id', 'seccion_id'])


## Upload to Mapbox as a tileset

In [ ]:
import os
os.environ['MAPBOX_ACCESS_TOKEN'] = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'


In [ ]:
# !cd /home/matias/Documents/electoral/datos/geojson
# !export MAPBOX_ACCESS_TOKEN=sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA

!tilesets delete-source -f matuteiglesias2 votos_provs_pct
!tilesets delete-source -f matuteiglesias2 votos_dptos_pct
!tilesets delete-source -f matuteiglesias2 votos_circs_pct

!tilesets delete -f matuteiglesias2.votos_pct

Source deleted.
Source deleted.
Source deleted.
Tileset deleted.


In [ ]:
## HOGARES. 
# (REPEAT FOR PERSONAS, M24, ETC.)
!tilesets upload-source matuteiglesias2 votos_provs_pct ./../datos/geojson/votos_pct_prov.geojson
!tilesets upload-source matuteiglesias2 votos_dptos_pct ./../datos/geojson/votos_pct_secc.geojson
# !tilesets upload-source matuteiglesias2 votos_circs_pct ./../datos/geojson/votos_pct_circ_lite.geojson
!tilesets upload-source matuteiglesias2 votos_circs_pct ./../datos/geojson/votos_pct_circ.geojson


# # tilesets create username.hello-world-tiles --recipe hello-world-recipe.json --name "hello world"
!tilesets create matuteiglesias2.votos_pct --recipe ./../datos/geojson/votos_pct-recipe.json --name "Votos - Porcentaje"
!tilesets publish matuteiglesias2.votos_pct

upload progress  [-------------------------------------------------------]    0%
{"id": "mapbox://tileset-source/matuteiglesias2/votos_provs_pct", "files": 1, "source_size": 1213716, "file_size": 1213716}
upload progress  [-------------------------------------------------------]    0%
{"id": "mapbox://tileset-source/matuteiglesias2/votos_dptos_pct", "files": 1, "source_size": 21310587, "file_size": 21310587}
upload progress  [-------------------------------------------------------]    0%
{"id": "mapbox://tileset-source/matuteiglesias2/votos_circs_pct", "files": 1, "source_size": 19428295, "file_size": 19428295}
{"message": "Successfully created empty tileset matuteiglesias2.votos_pct. Publish your tileset to begin processing your data into vector tiles."}
{"message": "Processing matuteiglesias2.votos_pct", "jobId": "clhyls5wa006e08l84l5cgrce"}

✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/matuteiglesias2.votos_pct or run tilesets job matuteiglesias2.votos_pct clhyls

In [ ]:
xx

NameError: name 'xx' is not defined

In [ ]:
import os
os.environ['MAPBOX_ACCESS_TOKEN'] = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'


In [ ]:
# !cd /home/matias/Documents/electoral/datos/geojson
# !export MAPBOX_ACCESS_TOKEN=sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA

!tilesets delete-source -f matuteiglesias2 votos_provs
!tilesets delete-source -f matuteiglesias2 votos_dptos
!tilesets delete-source -f matuteiglesias2 votos_circs

!tilesets delete -f matuteiglesias2.votos

Source deleted.
Source deleted.
Source deleted.
Tileset deleted.
Traceback (most recent call last):
  File "/home/matias/anaconda3/lib/python3.9/site-packages/cligj/features.py", line 192, in coords_from_query
    coords = json.loads(query)
  File "/home/matias/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/matias/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/matias/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/matias/anaconda3/bin/tilesets", line 8, in <module>
    sys.exit(cli())
  File "/home/matias/anaconda3/lib/python3.9/site-packages/click/core.py", line 11

In [ ]:
## HOGARES. 
# (REPEAT FOR PERSONAS, M24, ETC.)
!tilesets upload-source matuteiglesias2 votos_provs ./../datos/geojson/votos_prov.geojson
!tilesets upload-source matuteiglesias2 votos_dptos ./../datos/geojson/votos_secc.geojson
!tilesets upload-source matuteiglesias2 votos_circs ./../datos/geojson/votos_circ_lite.geojson


# # tilesets create username.hello-world-tiles --recipe hello-world-recipe.json --name "hello world"
!tilesets create matuteiglesias2.votos --recipe ./../datos/geojson/votos-recipe.json --name "Votos - Cantidad"
!tilesets publish matuteiglesias2.votos

upload progress  [-------------------------------------------------------]    0%
{"id": "mapbox://tileset-source/matuteiglesias2/votos_dptos", "files": 1, "source_size": 3824021, "file_size": 3824021}
upload progress  [-------------------------------------------------------]    0%
{"id": "mapbox://tileset-source/matuteiglesias2/votos_circs", "files": 1, "source_size": 1800597, "file_size": 1800597}
{"message": "Successfully created empty tileset matuteiglesias2.votos. Publish your tileset to begin processing your data into vector tiles."}
{"message": "Processing matuteiglesias2.votos", "jobId": "clhv5ddrx001f08laa4m60ts7"}

✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/matuteiglesias2.votos or run tilesets job matuteiglesias2.votos clhv5ddrx001f08laa4m60ts7 to view the status of your tileset.
